In [1]:
import gymnasium as gym
import highway_env
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.noise import NormalActionNoise

# Create the environment
env = gym.make("highway-v1")

# Wrap the environment
env.reset()

(array([[ 1.        ,  0.9155829 ,  0.75      ,  0.3125    ,  0.        ],
        [ 1.        ,  0.11780711, -0.25      , -0.02102297,  0.        ],
        [ 1.        ,  0.23142113, -0.75      , -0.03086345,  0.        ],
        [ 1.        ,  0.34464368, -0.25      , -0.02393701,  0.        ],
        [ 1.        ,  0.45102248, -0.25      , -0.03796508,  0.        ]],
       dtype=float32),
 {'speed': 25,
  'crashed': False,
  'action': 3,
  'rewards': {'collision_reward': 0.0,
   'right_lane_reward': 1.0,
   'high_speed_reward': 0.8333333333333334,
   'on_road_reward': 1.0,
   'reverse_reward': 0.0,
   'stopping_penalty': 0}})

In [2]:
# Create the SAC model
model = PPO(
    "MlpPolicy",
    env,
    learning_rate=0.0003,
    policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    batch_size=64,
    gamma=0.9,
    verbose=1,
    tensorboard_log="ppo_highway_tensorboard"
)

# Train the model
model.learn(total_timesteps=20000,tb_log_name = 'PPO_1')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\quynh\AppData\Local\Programs\Python\Python312\Lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Logging to ppo_highway_tensorboard\PPO_1_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.9     |
|    ep_rew_mean     | 10.2     |
| time/              |          |
|    fps             | 3        |
|    iterations      | 1        |
|    time_elapsed    | 561      |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 20.1        |
|    ep_rew_mean          | 16.3        |
| time/                   |             |
|    fps                  | 3           |
|    iterations           | 2           |
|    time_elapsed         | 1130        |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016249381 |
|    clip_fraction        | 0.261       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.59       |
|    explained_variance   | 0

In [4]:
model.learn(total_timesteps=20000,tb_log_name = 'PPO_1',reset_num_timesteps=False)

Logging to ppo_highway_tensorboard\PPO_1_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 38.3     |
|    ep_rew_mean     | 33.4     |
| time/              |          |
|    fps             | 4        |
|    iterations      | 1        |
|    time_elapsed    | 485      |
|    total_timesteps | 22528    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 37.6        |
|    ep_rew_mean          | 33          |
| time/                   |             |
|    fps                  | 4           |
|    iterations           | 2           |
|    time_elapsed         | 974         |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.029495802 |
|    clip_fraction        | 0.206       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.11       |
|    explained_variance   | 0

In [3]:
# Save the trained model
model.save("PPO_highwayv1")